In [15]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Administrator\Desktop\dev\tcc\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [16]:
import math
import os
import timeit
import random
import heapq
import threading
import networkx as nx

In [17]:
os.environ["INITIAL_GRAPH"]="004"
os.environ["CONTAMINANTS"]="2"
os.environ["LENGTH_SAMPLE"]="1000"
os.environ["STOP_ON_FIRST_BEST_SAMPLE"]="True"
os.environ["FLEXIBLE_BINARY_SEARCH"]="True"
os.environ["WITH_WEIGHT"]="True"
os.environ["VELOCITY"]="2"
os.environ["PARALLEL"]="False"
os.environ["MAX_PARALLEL"] = "8"
os.environ["ONE_IN"] = "10"
os.environ["FILE_INPUT_EXTENSION"] = "tgf"

In [18]:
class Hull:
    def __init__(self, array = None):
        self.infection = {} # mostra por quais vertices um vertice foi contaminado
        self.hull = []
        self.weights = []
        self.time = 0
        self.times = {} # tempo em houve a entrada do vertice
        self.times[self.time] = array or []
        for v in array or []:
            self.infection[v] = None
            self.hull.append(v)

    def append(self, other):
        self.hull.append(other)
        self.times[self.time].append(other)

    def append_with_weight(self, other, weight):
        self.hull.append(other)
        self.weights.append(weight)

    def __add__(self, other):
        return Hull(self.hull + other.hull)

    def __len__(self):
        return len(self.hull)
    
    def __contains__(self, key):
        return key in self.hull

    def __iter__(self):
        for v in self.hull:
            yield v

    def weighted_selection_without_replacement(self, n):
        # https://colab.research.google.com/drive/14Vnp-5xRHLZYE_WTczhpoMW2KdC6Cnvs#scrollTo=wEwWxLMKbpZn
        elt = [(math.log(random.random()) / self.weights[i], i) for i in range(len(self.weights))]
        return [x[1] for x in heapq.nlargest(n, elt)]

    def random_subset(self, n, with_weight = False):
        if with_weight:
            indexes = self.weighted_selection_without_replacement(n)
        else:
            indexes = random.sample(range(len(self.hull)), n)
        sample = [self.hull[i] for i in indexes]
        return Hull(sample), indexes

    def update_weights(self, indexes, internal = False):
        if internal:
            for i in indexes:
                self.weights[i] *= int(os.getenv('VELOCITY')) 
        else:
            sum_indexes_weights = sum(self.weights[i] for i in indexes)
            sum_non_indexes = sum(weight for weight in self.weights)
            remain = (((int(os.getenv('ONE_IN')) * sum_non_indexes) - sum_indexes_weights) + len(indexes)) // len(indexes)
            for i in indexes:
                self.weights[i] += remain
        biggest = max(self.weights)
        maximum = 1000000
        minimum = 1/10000000000
        if biggest > maximum: # normalize weights
            self.weights = [max(weight * maximum / biggest, minimum) for weight in self.weights]

    def evolve(self, array):
        if array:
            self.time += 1
            self.times[self.time] = array
            self.hull += array

    def last_border(self):
        return self.times[self.time]

    """
    contaminado na chave "i" tem a lista de vértices que o contaminaram
    os contaminados já no fecho inicial não tem uma lista e sim são iguais a None
    retorna True se for um novo contaminado por 2 elementos
    retorna False caso contrário
    """
    def infect(self, vcontaminant, vcontaminated):
        if vcontaminated in self.infection:
            if self.infection[vcontaminated] is None:
                return False
            if len(self.infection[vcontaminated]) < int(os.getenv('CONTAMINANTS')) and vcontaminant not in self.infection[vcontaminated]:
                self.infection[vcontaminated].add(vcontaminant)
                if len(self.infection[vcontaminated]) == int(os.getenv('CONTAMINANTS')):
                    return True
        else:
            self.infection[vcontaminated] = set()
            self.infection[vcontaminated].add(vcontaminant)
            if len(self.infection[vcontaminated]) == int(os.getenv('CONTAMINANTS')):
                # só existe essa condição para no futuro podermos evoluir pra qualquer N >= 1, aqui no caso N=2
                return True
        return False

    def initial_hull(self):
        return self.times[0]

    def write(self, graph, path):
        g = nx.Graph(graph.graph)
        for i in range(graph.vmin, graph.vmax + 1):
            g.add_node(i)
        for time, array in self.times.items():
            for i in array:
                g.nodes[i]['Time'] = time
        nx.write_gexf(g, f"outputs/hull_{path}.gexf")

In [19]:
class Graph:
    def __init__(self, path):
        self.reset_graph()
        self.read(path)
        self.avl_hull = None
        self.mnd_hull = None
        self.mandatory_hull() # set mandatory hull previously
        self.available_hull() # set available hull previously
        # self.write_graph(path)

    def reset_graph(self):
        self.graph = {}
        self.nedges = 0
        self.vmax = 0
        self.vmin = math.inf

    # def write_graph(self, path):
    #     dicts = []
    #     with open(f"inputs/{path}.txt") as f:
    #         while True:
    #             row = f.readline()
    #             if not row:
    #                 break
    #             v1, v2 = int(row.split()[0]), int(row.split()[1])
    #             dicts.append({'Source': v1, 'Target': v2, 'Type': 'Undirected'})
    #     with open(f"outputs/edges_{path}.csv", 'w', newline='') as output_file:
    #         dict_writer = csv.DictWriter(output_file, dicts[0].keys())
    #         dict_writer.writeheader()
    #         dict_writer.writerows(dicts)

    def read(self, path):
        self.path = f"inputs/{path}.{os.getenv('FILE_INPUT_EXTENSION')}"
        with open(self.path) as f:
            while True:
                row = f.readline()
                if not row:
                    break
                self.nedges += 1
                if "#" in row:
                    print('encontrado um #')
                    self.reset_graph()
                    continue
                v1, v2 = int(row.split()[0]), int(row.split()[1])
                self.vmin = min(self.vmin, v1, v2)
                self.vmax = max(self.vmax, v1, v2)
                self.add_on_adjacenty_list_undirected(v1, v2)

    def __len__(self):
        # o numero de vertices do grafo
        # obs.: vertices nao encontrados na entrada (entre vmin e vmax) sao considerados como vertices isolados de grau 0
        return self.vmax

    def add_on_adjacenty_list_undirected(self, u, w):
        self.add_on_adjacency_list(u, w)
        self.add_on_adjacency_list(w, u)

    def add_on_adjacency_list(self, u, w):
        if u not in self.graph:
            self.graph[u] = set()
            self.graph[u].add(w)
        else:
            self.graph[u].add(w)

    # @functools.lru_cache
    def mandatory_hull(self):
        if self.mnd_hull is None:
            # conjunto com o vertices que necessariamente devem estar no fecho inicial pois de outra forma nao seriam contaminados
            hull = Hull()
            for i in range(self.vmin, self.vmax + 1):
                if i not in self.graph: # vertice de grau 0
                    hull.append(i)
                elif len(self.graph[i]) < int(os.getenv('CONTAMINANTS')): # vertices de grau mais baixo que o numero de vizinhos necessarios para contaminar
                    hull.append(i)
            self.mnd_hull = hull
        return self.mnd_hull

    # @functools.lru_cache
    def available_hull(self):
        if self.avl_hull is None:
            # conjunto de vertices que podem ser selecionados para serem parte de um hull inicial
            hull = Hull()
            for i in range(self.vmin, self.vmax + 1):
                if i not in self.mandatory_hull():
                    hull.append_with_weight(i, 1)
            self.avl_hull = hull
        return self.avl_hull

    def evolve_hull(self, hull):
        hullarray = []
        for v in hull.last_border():
            if v in self.graph:
                for w in self.graph[v]:
                    wasinfected = hull.infect(v, w)
                    if wasinfected:
                        hullarray.append(w)
        hull.evolve(hullarray)
        return hull

    def hull_algorithm(self, hull):
        last_hull_length = len(hull)
        while True:
            # print("t: {}, fecho: {}".format(t, fecho))
            hull = self.evolve_hull(hull)
            if len(hull) == last_hull_length:
                break
            else:
                last_hull_length = len(hull)
            # print("novo_fecho: {}".format(novo_fecho))
        return hull

In [20]:
def run_samples(graph, n):
    first = True
    for cnt in range(0, int(os.getenv('LENGTH_SAMPLE'))):
        random_hull, idx = graph.available_hull().random_subset(n, os.getenv('WITH_WEIGHT') == 'True')
        hull = graph.mandatory_hull() + random_hull
        hull = graph.hull_algorithm(hull)
        if first or (len(hull) > len(hull_best)) or (len(hull) == len(hull_best) and hull.time < hull_best.time):
            if not first and os.getenv('WITH_WEIGHT') == 'True':
                graph.available_hull().update_weights(indexes, True)
            first = False
            hull_best = hull
            indexes = idx
        if os.getenv('STOP_ON_FIRST_BEST_SAMPLE') == 'True' and reach_threshold(hull, len(graph)):
            break
    return hull_best, indexes


def reach_threshold(hull, vmax):
    return len(hull) == vmax


def optimize(graph, flexible = False):
    # when flexible is True run a reset on minimum to restart binary search, it is equivalent to run binary search multiple times
    minimum = 1
    maximum = len(graph.available_hull())
    n = math.ceil(maximum / 2)
    first_hull_time = True
    first_hull = True
    while True:
        print('minimum: {}, maximum: {}, n: {}'.format(minimum, maximum, n))
        
        if os.getenv('PARALLEL') == 'True':
            hull, indexes = run_samples_parallel(graph, n) # on hold for future
        else:
            hull, indexes = run_samples(graph, n)
        
        if reach_threshold(hull, len(graph)):
            if first_hull_time or (hull.time <= hull_time.time and len(hull.initial_hull()) < len(hull_time.initial_hull())):
                first_hull_time = False
                hull_time = hull
            if first_hull or len(hull.initial_hull()) < len(hull_best.initial_hull()) or (len(hull.initial_hull()) == len(hull_best.initial_hull()) and hull.time < hull_best.time):
                if not first_hull and os.getenv('WITH_WEIGHT') == 'True':
                    graph.available_hull().update_weights(indexes)
                first_hull = False
                hull_best = hull
                print("tamanho do MELHOR FECHO INICIAL: {}".format(len(hull_best.initial_hull())))
                print("numero de vertices alcancados pelo MELHOR FECHO INICIAL: {}".format(len(hull_best)))
                print("tempo do MELHOR FECHO INICIAL: {}".format(hull_best.time))
                print()
                if flexible:
                    minimum = 1
            maximum = n
            n = (maximum + minimum) // 2
        else:
            minimum = n
            n = n + math.ceil((maximum - minimum) / 2) # maximum - max(math.ceil((maximum - n) / 2), 1) # (maximum - n) // 2
        if maximum - minimum <= 1:
            break
    return hull_best, hull_time


def exec():
    graph = Graph(f"{os.getenv('INITIAL_GRAPH')}")
    start = timeit.default_timer()

    hull_best, hull_time = optimize(graph, os.getenv('FLEXIBLE_BINARY_SEARCH') == 'True')
    
    stop = timeit.default_timer()
    print(f'\nfinalizado em {stop - start} segundos\n')

    # print("vertices do MELHOR FECHO INICIAL: {}".format(hull_best))
    print("tamanho do MELHOR FECHO INICIAL: {}".format(len(hull_best.initial_hull())))
    print("numero de vertices alcancados pelo MELHOR FECHO INICIAL: {}".format(len(hull_best)))
    print("tempo do MELHOR FECHO INICIAL: {}".format(hull_best.time))
    print()

    # print("vertices do FECHO DE MELHOR TEMPO: {}".format(hull_time))
    print("tamanho do FECHO DE MELHOR TEMPO: {}".format(len(hull_time.initial_hull())))
    print("numero de vertices alcancados pelo MELHOR FECHO INICIAL: {}".format(len(hull_time)))
    print("tempo do FECHO DE MELHOR TEMPO: {}".format(hull_time.time))

    hull_best.write(graph, f"best_{os.getenv('INITIAL_GRAPH')}")
    hull_time.write(graph, f"time_{os.getenv('INITIAL_GRAPH')}")

def bulkexec():
    first = True
    for i in range(1, 5):
        graphname = str(i).zfill(3)
        graph = Graph(graphname)
        start = timeit.default_timer()
        hull_best, hull_time = optimize(graph, os.getenv('FLEXIBLE_BINARY_SEARCH') == 'True')
        stop = timeit.default_timer()
        exec_time = stop - start

        hull_best.write(graph, f"best_{graphname}")
        hull_time.write(graph, f"time_{graphname}")

        dicts = {
            'Graph': graphname, 
            'Time': int(exec_time),
            'Len': len(hull_best.initial_hull()),
            'Alcance': len(hull_best),
            'T': hull_best.time,
            'Len(hulltime)': len(hull_time.initial_hull()),
            'Alcance(hulltime)': len(hull_time),
            'T(hulltime)': hull_time.time,
            'INITIAL_GRAPH': os.getenv('INITIAL_GRAPH'), 
            'CONTAMINANTS': os.getenv('CONTAMINANTS'),
            'LENGTH_SAMPLE': os.getenv('LENGTH_SAMPLE'),
            'STOP_ON_FIRST_BEST_SAMPLE': os.getenv('STOP_ON_FIRST_BEST_SAMPLE'),
            'FLEXIBLE_BINARY_SEARCH': os.getenv('FLEXIBLE_BINARY_SEARCH'),
            'WITH_WEIGHT': os.getenv('WITH_WEIGHT'),
            'VELOCITY': os.getenv('VELOCITY'),
            'PARALLEL': os.getenv('PARALLEL'),
            'MAX_PARALLEL': os.getenv('MAX_PARALLEL'),
            'ONE_IN': os.getenv('ONE_IN')
        }
        with open(f"outputs/results.csv", 'a', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, dicts.keys())
            if first:
                dict_writer.writeheader()
            dict_writer.writerow(dicts)
            first = False

In [21]:
exec()
# bulkexec()

minimum: 1, maximum: 621, n: 311
minimum: 311, maximum: 621, n: 466
tamanho do MELHOR FECHO INICIAL: 845
numero de vertices alcancados pelo MELHOR FECHO INICIAL: 1000
tempo do MELHOR FECHO INICIAL: 5

minimum: 1, maximum: 466, n: 233
minimum: 233, maximum: 466, n: 350
minimum: 350, maximum: 466, n: 408
minimum: 408, maximum: 466, n: 437
tamanho do MELHOR FECHO INICIAL: 816
numero de vertices alcancados pelo MELHOR FECHO INICIAL: 1000
tempo do MELHOR FECHO INICIAL: 4

minimum: 1, maximum: 437, n: 219
minimum: 219, maximum: 437, n: 328
minimum: 328, maximum: 437, n: 383
tamanho do MELHOR FECHO INICIAL: 762
numero de vertices alcancados pelo MELHOR FECHO INICIAL: 1000
tempo do MELHOR FECHO INICIAL: 4

minimum: 1, maximum: 383, n: 192
minimum: 192, maximum: 383, n: 288
minimum: 288, maximum: 383, n: 336
tamanho do MELHOR FECHO INICIAL: 715
numero de vertices alcancados pelo MELHOR FECHO INICIAL: 1000
tempo do MELHOR FECHO INICIAL: 6

minimum: 1, maximum: 336, n: 168
minimum: 168, maximum: 